In [ ]:
!pip install scikit-learn

In [ ]:
!pip install evaluate
!pip install rouge-score

In [ ]:
import evaluate
rouge = evaluate.load('rouge')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**data processing**

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/AI LAB PROJECT/train.csv", usecols=['highlights', 'article'], nrows=5000)

In [ ]:
df

,article,highlights
0,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...
...,...,...
4995,"(CNN) -- Despite being recently branded a ""rot...",Mario Balotelli returns to Italy joining AC Mi...
4996,London (CNN) -- Let's play spot the villain. O...,Several priests have recently told kids that S...
4997,A jealous husband who tried to strangle his nu...,Royston Jones was convinced that wife Claire J...
4998,Defence bosses were last night embroiled in a ...,Author 'forced to resign' from the Army after ...


In [ ]:
df = df.rename(columns={"highlights":"target_text", "article":"source_text"})
df = df[['source_text', 'target_text']]

In [ ]:
df.head()

,source_text,target_text
0,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...


In [ ]:
df['source_text'] = "summarize: " + df['source_text']
df.head()

,source_text,target_text
0,summarize: By . Associated Press . PUBLISHED: ...,"Bishop John Folda, of North Dakota, is taking ..."
1,summarize: (CNN) -- Ralph Mata was an internal...,Criminal complaint: Cop used his role to help ...
2,summarize: A drunk driver who killed a young w...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,summarize: (CNN) -- With a breezy sweep of his...,Nina dos Santos says Europe must be ready to a...
4,summarize: Fleetwood are the only team still t...,Fleetwood top of League One after 2-0 win at S...


In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2)
train_df.shape, test_df.shape

((4000, 2), (1000, 2))

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

**Google T5 model tuning**

In [ ]:
! pip install simplet5 -q

In [ ]:
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")

In [ ]:
size=train_df['source_text'].size
size

4000

In [ ]:
model.train(train_df=train_df[:4000],
            eval_df=test_df[:100],
            source_max_token_len=512,
            target_max_token_len=150,
            outputdir="/content/drive/MyDrive/Colab Notebooks/Model",
            batch_size=16, max_epochs=8, use_gpu=True)

INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
INFO:pytorch_lightning.utilities.seed:Global seed set to 42
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
#! ( cd outputs; ls )

In [ ]:
model.load_model("t5","/content/drive/MyDrive/Colab Notebooks/Model/simplet5-epoch-7-train-loss-0.9762-val-loss-1.6444", use_gpu=True)

**Youtube video to text processing**

In [ ]:
pip install deepmultilingualpunctuation

In [ ]:
from deepmultilingualpunctuation import PunctuationModel

In [ ]:
model = PunctuationModel()

In [ ]:
!pip install youtube-transcript-api

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi

In [ ]:
link = "https://www.youtube.com/watch?v=veMFCFyOwFI"
id = link.split("=")[-1]
id

'veMFCFyOwFI'

In [ ]:
pretext=YouTubeTranscriptApi.get_transcript(id)
pretext

[{'text': 'The Middle East is one of the most complex\nregions in the world:',
  'start': 2.429,
  'duration': 2.521},
 {'text': 'Currently there are 4 failing states and 3\nwars, with major powers increasingly taking',
  'start': 4.95,
  'duration': 5.4},
 {'text': 'opposite sides.', 'start': 10.35, 'duration': 1.65},
 {'text': 'Countless armed militias and terrorist groups\nare spreading violence across borders.',
  'start': 12.0,
  'duration': 4.08},
 {'text': 'The region has seen conflict after conflict\ngoing back well into the 20th century.',
  'start': 16.08,
  'duration': 4.41},
 {'text': 'But among all the uprisings, civil wars, and\ninsurgencies, two countries always seem to',
  'start': 20.49,
  'duration': 4.15},
 {'text': 'be involved: Saudi Arabia and Iran.',
  'start': 24.64,
  'duration': 2.97},
 {'text': 'They’re bitter rivals, and their feud is\nthe key to understanding conflicts in the',
  'start': 27.61,
  'duration': 4.11},
 {'text': 'Middle East.', 'start': 31.72,

In [ ]:
text=""
for i in pretext:
  s=i["text"]
  text=text+s+" "
text

"The Middle East is one of the most complex\nregions in the world: Currently there are 4 failing states and 3\nwars, with major powers increasingly taking opposite sides. Countless armed militias and terrorist groups\nare spreading violence across borders. The region has seen conflict after conflict\ngoing back well into the 20th century. But among all the uprisings, civil wars, and\ninsurgencies, two countries always seem to be involved: Saudi Arabia and Iran. They’re bitter rivals, and their feud is\nthe key to understanding conflicts in the Middle East. The Saudis and Iranians have never actually\ndeclared war on each other. Instead, they fight indirectly by supporting\nopposing sides in other countries and inciting conflicts. This is known as proxy warfare. And it’s had a devastating effect on the\nregion. Countries, especially poor ones, can’t function if there are larger countries pulling strings within their borders. Both the Saudis and the Iranians, see these civil wars as both

In [ ]:
text=model.restore_punctuation(text)
text

In [ ]:
finaltext="summarize:"+text
finaltext


"summarize:The Middle East is one of the most complex\nregions in the world: Currently there are 4 failing states and 3\nwars, with major powers increasingly taking opposite sides. Countless armed militias and terrorist groups\nare spreading violence across borders. The region has seen conflict after conflict\ngoing back well into the 20th century. But among all the uprisings, civil wars, and\ninsurgencies, two countries always seem to be involved: Saudi Arabia and Iran. They’re bitter rivals, and their feud is\nthe key to understanding conflicts in the Middle East. The Saudis and Iranians have never actually\ndeclared war on each other. Instead, they fight indirectly by supporting\nopposing sides in other countries and inciting conflicts. This is known as proxy warfare. And it’s had a devastating effect on the\nregion. Countries, especially poor ones, can’t function if there are larger countries pulling strings within their borders. Both the Saudis and the Iranians, see these civil wa

**Google T5 Abstractive Summarization Result**

In [ ]:
t5_result=model.predict(finaltext)
t5_result[0]

'The Middle East is one of the most complex regions in the world. There are 4 failing states and 3 wars, with major powers increasingly taking opposite sides. Saudi Arabia and Iran have never declared war on each other. They fight indirectly by supporting opposing sides in other countries.'

**Saving Model**

In [ ]:
import pickle
with open('/content/drive/MyDrive/AI LAB PROJECT/model.pkl', 'wb') as f:
    pickle.dump(model, f)


**SpaCy Preprocessing**

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [ ]:
doc = nlp(text)

In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS
stopwords = list(STOP_WORDS)

In [ ]:
from string import punctuation
punctuation = punctuation + '\n'

In [ ]:
wordfreq = {}
for word in doc:
  if word.text.lower() not in stopwords:
    if word.text.lower() not in punctuation:
      if word.text not in wordfreq.keys():
        wordfreq[word.text] = 1
      else:
        wordfreq[word.text] += 1

In [ ]:
for word in wordfreq.keys():
  wordfreq[word] = wordfreq[word]/max(wordfreq.values())
wordfreq

{'Middle': 0.23333333333333334,
 'East': 0.23333333333333334,
 'complex': 0.03333333333333333,
 'regions': 0.03333333333333333,
 'world': 0.23333333333333334,
 'Currently': 0.03333333333333333,
 '4': 0.03333333333333333,
 'failing': 0.03333333333333333,
 'states': 0.03333333333333333,
 '3': 0.03333333333333333,
 'wars': 0.26666666666666666,
 'major': 0.03333333333333333,
 'powers': 0.03333333333333333,
 'increasingly': 0.03333333333333333,
 'taking': 0.03333333333333333,
 'opposite': 0.03333333333333333,
 'sides': 0.1,
 'Countless': 0.03333333333333333,
 'armed': 0.06666666666666667,
 'militias': 0.2,
 'terrorist': 0.06666666666666667,
 'groups': 0.3,
 'spreading': 0.03333333333333333,
 'violence': 0.03333333333333333,
 'borders': 0.06666666666666667,
 'region': 0.13333333333333333,
 'seen': 0.03333333333333333,
 'conflict': 0.1,
 'going': 0.03333333333333333,
 '20th': 0.03333333333333333,
 'century': 0.06666666666666667,
 'uprisings': 0.03333333333333333,
 'civil': 0.23333333333333334

In [ ]:
senttokens = [i for i in doc.sents]
senttokens

[The Middle East is one of the most complex
 regions in the world: Currently there are 4 failing states and 3
 wars, with major powers increasingly taking opposite sides.,
 Countless armed militias and terrorist groups
 are spreading violence across borders.,
 The region has seen conflict after conflict
 going back well into the 20th century.,
 But among all the uprisings, civil wars, and
 insurgencies, two countries always seem to be involved: Saudi Arabia and Iran.,
 They’re bitter rivals, and their feud is
 the key to understanding conflicts in the Middle East.,
 The Saudis and Iranians have never actually
 declared war on each other.,
 Instead, they fight indirectly by supporting
 opposing sides in other countries and inciting conflicts.,
 This is known as proxy warfare.,
 And it’s had a devastating effect on the
 region.,
 Countries, especially poor ones, can’t function if there are larger countries pulling strings within their borders.,
 Both the Saudis and the Iranians, see thes

In [ ]:
sentweight = {}
for sent in senttokens:
  for word in sent:
    if word.text.lower() in wordfreq.keys():
      if sent not in sentweight.keys():
        sentweight[sent] = wordfreq[word.text.lower()]
      else:
        sentweight[sent] += wordfreq[word.text.lower()]
sentweight

{The Middle East is one of the most complex
 regions in the world: Currently there are 4 failing states and 3
 wars, with major powers increasingly taking opposite sides.: 0.9666666666666666,
 Countless armed militias and terrorist groups
 are spreading violence across borders.: 0.7666666666666666,
 The region has seen conflict after conflict
 going back well into the 20th century.: 0.5,
 But among all the uprisings, civil wars, and
 insurgencies, two countries always seem to be involved: Saudi Arabia and Iran.: 0.9333333333333332,
 They’re bitter rivals, and their feud is
 the key to understanding conflicts in the Middle East.: 0.6666666666666666,
 The Saudis and Iranians have never actually
 declared war on each other.: 0.8666666666666667,
 Instead, they fight indirectly by supporting
 opposing sides in other countries and inciting conflicts.: 1.4333333333333333,
 This is known as proxy warfare.: 0.5333333333333333,
 And it’s had a devastating effect on the
 region.: 0.26666666666666

**SpaCy Extractive Summarization Result**

In [ ]:
from heapq import nlargest
sizee=int(len(senttokens)*0.05)
summ=nlargest(sizee, sentweight, key=sentweight.get)
summ

[Both the Iranians and the Saudis feel that their vital national interests, are threatened, are in jeopardy, because of different things happening in these civil
 wars, things they blame each other for.,
 They want the region stable, and they don't want anbody rising up and overthrowing a sclerotic, autocratic government, for fear that it might inspire their own people to do the same.,
 The Iranians are the ultimate anti-status quo power, they have been trying for decades to overturn the regional order.,
 That trend continued into the Arab Spring,
 a series of anti-monarchy, pro-democracy protests that swept through the Middle East in 2011.,
 The Saudi government is threatening Qatar, a tiny Gulf state that was developing ties with Iran.]

In [ ]:
spacy_res = [word.text for word in summ]
print(spacy_res)

['Both the Iranians and the Saudis feel that their vital national interests, are threatened, are in jeopardy, because of different things happening in these civil\nwars, things they blame each other for.', "They want the region stable, and they don't want anbody rising up and overthrowing a sclerotic, autocratic government, for fear that it might inspire their own people to do the same.", 'The Iranians are the ultimate anti-status quo power, they have been trying for decades to overturn the regional order.', 'That trend continued into the Arab Spring,\na series of anti-monarchy, pro-democracy protests that swept through the Middle East in 2011.', 'The Saudi government is threatening Qatar, a tiny Gulf state that was developing ties with Iran.']


In [ ]:
spacy_result = ' '.join(spacy_res)
print(spacy_result)

Both the Iranians and the Saudis feel that their vital national interests, are threatened, are in jeopardy, because of different things happening in these civil
wars, things they blame each other for. They want the region stable, and they don't want anbody rising up and overthrowing a sclerotic, autocratic government, for fear that it might inspire their own people to do the same. The Iranians are the ultimate anti-status quo power, they have been trying for decades to overturn the regional order. That trend continued into the Arab Spring,
a series of anti-monarchy, pro-democracy protests that swept through the Middle East in 2011. The Saudi government is threatening Qatar, a tiny Gulf state that was developing ties with Iran.


**Evaluation**

In [ ]:
references = ["The Saudis and Iranians have never actually declared war on each other. Instead, they fight indirectly by supporting opposing sides in other countries and inciting conflicts. This is known as proxy warfare. And it’s had a devastating effect on the region. Countries, especially poor ones, can’t function if there are larger countries pulling strings within their borders. And that’s exactly what's happening in the Middle East. The Saudi-Iranian rivalry has become a fight over influence, and the whole region is a battlefield."]

In [ ]:
predictions_t5 = [t5_result[0]]
results_t5 = rouge.compute(predictions=predictions_t5, references=references)
print(results_t5)

{'rouge1': 0.42647058823529416, 'rouge2': 0.26865671641791045, 'rougeL': 0.27941176470588236, 'rougeLsum': 0.27941176470588236}


In [ ]:
predictions_spacy = [spacy_result]
results_spacy = rouge.compute(predictions=predictions_spacy, references=references)
print(results_spacy)

{'rouge1': 0.2857142857142857, 'rouge2': 0.07692307692307691, 'rougeL': 0.1714285714285714, 'rougeLsum': 0.20952380952380953}
